In [1]:
from function.function import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from pandas.core import datetools
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import warnings

warnings.filterwarnings('ignore')
get_ipython().magic('matplotlib inline')

ModuleNotFoundError: No module named 'function'

## 1.EDA

### 1-1 데이터 소개

In [ ]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [ ]:
print("Train data : ", train.shape)
print("Test  data : ", test.shape)

In [ ]:
train.tail()

In [ ]:
test.tail()

In [ ]:
train.describe()

Find : 
1. LotFrontage, MasVnrArea 등 일부 데이터가 missing되었음

### 1-2 타겟 데이터 (SalePrice)

In [ ]:
train["SalePrice"].describe()

In [ ]:
plt.figure(figsize=(17,6))

plt.subplot(131)
sns.distplot(train["SalePrice"])

plt.subplot(132)
stats.probplot(train["SalePrice"], plot=plt)

plt.subplot(133)
sns.boxplot(train["SalePrice"])
plt.tight_layout()
plt.show()

Find :  
1. 쏠림현상이 있음.
2. log를 사용하여 정규화 작업이 필요함
3. 이상치 처리가 필요함

### 1-3  Analysis the univariance

In [ ]:
train["MSSubClass"] = train["MSSubClass"].astype('str')

In [ ]:
categorical_columns = ["ExterQual", "ExterCond","BsmtQual", "BsmtCond","BsmtFinType1","BsmtFinType2","HeatingQC","KitchenQual","FireplaceQu","GarageQual","GarageCond","PoolQC"]

for c in categorical_columns:
    train[c][(train[c] == "Ex") | (train[c] == "GLQ")] = 10
    train[c][(train[c] == "Gd") | (train[c] == "ALQ")] = 8
    train[c][(train[c] == "TA") | (train[c] == "BLQ")] = 6
    train[c][(train[c] == "Fa") | (train[c] == "Rec")] = 4
    train[c][(train[c] == "Po") | (train[c] == "LwQ")] = 2
    train[c][(train[c] == "NA") | (train[c] == "NaN") |(train[c] == "Unf")] = 0
    train[c] = train[c].astype('float')

In [ ]:
# Categoric and Numerical 변수 분리
numerical_features = []
categorical_features = []
for f in train.columns:
    if train.dtypes[f] != 'object':
        numerical_features.append(f)
    else:
        categorical_features.append(f)

In [ ]:
print("Numerical Features Qty :", len(numerical_features),"\n")
print("Numerical Features : ", numerical_features, "\n\n")
print("Categorical Features Qty :", len(categorical_features),"\n")
print("Categorical Features :", categorical_features)

#### Numerical Features with SalePrice

In [ ]:
fig, ax = plt.subplots(8,6, figsize = (20,20))

for idx, n in enumerate(numerical_features):
    if n == 'SalePrice':
        continue
    sns.regplot(x=n, y='SalePrice', data=train, ax = ax[idx//6,idx%6])
    ax[idx//6, idx % 6].set(yticklabels=[])
    ax[idx//6, idx % 6].set(xticklabels=[])
    
    continue    

In [ ]:
fig, ax = plt.subplots(6,6, figsize = (20,20))

for idx, n in enumerate(categorical_features):
    
    sns.countplot(x=n, data=train, ax = ax[idx//6, idx % 6])
    ax[idx//6, idx % 6].set(yticklabels=[])
    
    continue       

## 2. Feature Engineering

### 2-1 SalePrice와 변수들 간의 관계 및 이상치 제거

#### 2-1-1 GrLivArea  &  2ndFloor & Basement

In [ ]:
df_train = train.copy()
df_train["2ndFloor"] = "2ndFloor"
df_train["2ndFloor"].loc[df_train["2ndFlrSF"]==0] = "No 2ndFloor"
df_train["Basement"] = "Basement"
df_train["Basement"].loc[df_train["TotalBsmtSF"]==0] = "No Basement"

In [ ]:
grid = sns.jointplot(x = "GrLivArea", y = "SalePrice", data=train, kind="reg")
grid.fig.set_size_inches(15,5)

plt.figure(figsize = (20,8))
plt.subplot(211)
g = sns.stripplot(x = "GrLivArea", y = 'SalePrice', hue = "2ndFloor", data = df_train, alpha = 0.7)
g.set_xlabel('GrLivArea')
g.set_ylabel('SalePrice')
g.set_ylim(0,900000)
g.set_xticks([])
g.set_title('GrLiv & 2ndFloor - SalePrice')

plt.subplot(212)
b = sns.stripplot( x = "GrLivArea", y = 'SalePrice', hue = "Basement", data = df_train, alpha = 0.7)
b.set_xlabel('GrLivArea')
b.set_ylabel('SalePrice')
b.set_ylim(0,900000)
b.set_title('GrLivArea & Basement - SalePrice')
b.set_xticks([])

plt.show()

In [ ]:
outlier_GrLivArea = train[(train["GrLivArea"] >4000)]
outlier_GrLivArea

In [ ]:
train.drop(outlier_GrLivArea.index, axis=0,inplace=True)

Find : 1. Grliv 넓이가 중간 아래부분까지는 같은 넓이의 집은 2층보다 1층집의 값 더 비싼것을 알수 있었다
       2. 베이스먼트가 없는집은 거의 없으며, 집값에는 영향이 없는것 같다 

#### 2-1-2 OverallQual & OverallCond

In [ ]:
plt.figure(figsize=(15,8))

ax1 = plt.subplot2grid((2,2), (0,0), colspan = 2)
for qual in range(1,10+1):
    index = train.OverallQual == qual
    ax1.scatter(train.GrLivArea.loc[index], train.SalePrice.loc[index], data=train, label= qual, alpha =0.5)
ax1.legend(loc = 0)
ax1.set_title("GrLivArea & OverallQual - SalePrice")

ax2 = plt.subplot2grid((2,2), (1,0))
sns.boxplot(x = "OverallQual", y = "SalePrice", data=train, ax= ax2)
ax2.set_title('OverallQual - SalePrice')

ax3 = plt.subplot2grid((2,2), (1,1))
sns.boxplot(x = "OverallCond", y = "SalePrice", data=train, ax= ax3)
ax3.set_title('OverallCond - SalePrice')

In [ ]:
outlier_OverallQual = train[(train["SalePrice"] >600000) | ((train["SalePrice"] > 500000) & (train["OverallQual"]==9)) | \
      ((train["SalePrice"] > 500000) & ((train["OverallQual"]==9) |(train["OverallQual"]==8)))]
outlier_OverallQual

In [ ]:
train.drop(outlier_OverallQual.index, axis=0, inplace=True)

Find : 1. 품질과 집값은 선형관계를 발견할 수 있으나 overall condition과 집값은 관계가 없는것을 발견할 수 있었음

#### 2-1-2 GarageArea & GarageCars

In [ ]:
outlier_GarageArea = train[(train["SalePrice"] >500000)]
outlier_GarageArea

In [ ]:
train.drop(outlier_GarageArea.index, axis=0,inplace=True)

Find :

#### 2-1-3 Neighborhood

In [ ]:
Neighbor = train.pivot_table(index="Neighborhood",values="SalePrice", aggfunc='mean').sort_values(by = ["SalePrice"], ascending = False)
Neighbor = Neighbor.reset_index()

g = sns.factorplot(x = "Neighborhood", y="SalePrice", data=Neighbor, size =8, kind="bar")
g.set_xticklabels(rotation=45)
g.fig.set_size_inches(15,5)
plt.show()

In [ ]:
df_train = train.copy()

# High_price_neighbor : 판매가 250000 이상 
# Middle_price_neighbor : 판매가 150000 ~ 250000
# Low_price_neighbor : 판매가 150000 이하
def neighbor_level(x):
    High_price_neighbor = ['NoRidge','NridgHt','StoneBr'] 
    Middle_price_neighbor = ['Timber','Somerst','Veenker','ClearCr','Crawfor','NWAmes', 'Gilbert','Blmngtn', 'SWISU','Mitchel','CollgCr']
    Low_price_neighbor = ['IDOTRR','Blueste', 'Sawyer','NAmes', 'BrDale', 'OldTown','MeadowV', 'NPkVill','BrkSide','Edwards']
    
    if str(x) in High_price_neighbor:
        return "high"
    elif str(x) in Middle_price_neighbor:
        return "middle"
    elif str(x) in Low_price_neighbor:
        return "low"

In [ ]:
df_train["neighbor_level"] = df_train["Neighborhood"].apply(neighbor_level)
# l = df_train.groupby("neighbor_level").agg('mean')
# l = l.reindex(["high",'middle',"low"])

In [ ]:
fig, ax = plt.subplots(7,7, figsize = (20,20))

for idx, n in enumerate(numerical_features):

    sns.barplot(x="neighbor_level", y= n, data=df_train, ax = ax[idx//7,idx%7])
    ax[idx//7, idx % 7].set(yticklabels=[])
    ax[idx//7, idx % 7].set(xticklabels=["high","middle","low"])
    ax[idx//7, idx % 7].set_xlabel("")
    ax[idx//7, idx % 7].set_ylabel(n)
    continue  

Find:


#### 2-1-4 YearBuilt

In [ ]:
plt.figure(figsize=(15,6))
fig = sns.boxplot(x="YearBuilt", y="SalePrice", data=train)
fig.axis(ymin=0, ymax=800000)
plt.xticks(rotation=90)
plt.show()

plt.figure(figsize=(15,6))
ax2 = plt.subplot(1,2,1)
ax2.scatter(x = train['YearBuilt'], y = train['YearRemodAdd'], alpha = 0.5)
plt.xlabel('YearBuilt')
plt.ylabel('YearRemodAdd')
ax3 = plt.subplot(1,2,2)
ax3.scatter(x = train['YearBuilt'], y = train['YrSold'], alpha = 0.5)
plt.xlabel('YearBuilt')
plt.ylabel('Yearsold')
plt.show()

plt.figure(figsize=(15,6))
ax4 = plt.subplot2grid((2,2), (0,0), colspan = 2)
ax4.scatter(x = train['YearBuilt'], y = train['OverallQual'], alpha = 0.5)
plt.xlabel('YearBuilt')
plt.ylabel('OverallQual')
ax5 = plt.subplot2grid((2,2), (1,0), colspan = 2)
sns.countplot(x = "MoSold", data=train, ax = ax5)
plt.show()

Find

1. 1950년대 이후로 지어진 집부터는 대체로 최근에 지어진 집의 가격이 높음을 알 수 있음.
2. 아직 리모델링이 되지 않은 집은 지어진 년도와 같은 년도로 기록되어 있음.
3. 리모델링은 1950년도 이후에 시행되었으며 오래된 집들은 대부분 1950년도에 리모델링 되었다고 기록되어 있음.
4. 매매년도는 2006년부터 2010년까지의 기록으로 되어있으며, 매매가 가장 많았던 년도는 2009년임.
5. OverallQual이 8-10인 집은 대체로 1980년도 이후에 지어진 집임.
6. 날이 따뜻한 5, 6, 7월에 가장 매매가 활발하게 이루어짐.

#### 2-1-5 Fireplaces & FireplaceQu

In [ ]:
plt.figure(figsize=(15,10))
ax1 = plt.subplot2grid((2,2), (0,0), colspan = 2)
sns.stripplot(x = train['Fireplaces'], y = train['SalePrice'], alpha = 0.5, jitter = True, ax=ax1)

ax2 = plt.subplot2grid((2,2), (1,0))
sns.countplot(x = "FireplaceQu", data=train, ax = ax2, order = train["FireplaceQu"].value_counts().index)

ax3 = plt.subplot2grid((2,2), (1,1))
sns.boxplot(x = 'FireplaceQu', y = 'OverallQual', data = train, ax=ax3)
plt.show()

plt.figure(figsize=(15,10))
ax4 = plt.subplot(2,1,1)
sns.stripplot(x='FireplaceQu', y='SalePrice', hue='Fireplaces', data=train, jitter=True, alpha=0.6, ax=ax4)

ax5 = plt.subplot(2,1,2)
ax5.scatter(x = train['Fireplaces'], y = train['GrLivArea'], alpha = 0.5)
plt.xlabel('Fireplaces')
plt.ylabel('GrLivArea')
plt.show()


Find

1. FirePlaces가 0개인 집과 1개인 집의 가격 차이가 있음을 볼 수 있음.
2. FireplaceQu와 OverallQual은 밀접한 관계가 있음.
3. FireplaceQu는 SalePrice에 많은 영향을 미친다고 볼 수는 없음.
4. 같은 GrLivArea임에도 Fireplaces의 갯수는 차이가 있음.

#### 2-1-6 BsmtBath

BsmtFullBath : 지하실 가득한 욕실  
BsmtHalfBath : 지하 욕실 반 화장실

BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',

In [ ]:
plt.figure(figsize=(15,6))
sns.stripplot(x = "BsmtQual", y = "SalePrice", data=train, jitter=True)
plt.show()

In [ ]:
plt.figure(figsize=(15,6))

ax1 = plt.subplot(1,2,1)
for BSMT in range(0,5):
    index = train.BsmtFullBath == BSMT
    ax1.scatter(x = train.BsmtFullBath.loc[index], y = train.SalePrice.loc[index], data=train, label=BSMT, alpha='0.5')

ax1.legend()
ax1.set_title('BsmtFullBath - SalePrice')

ax2 = plt.subplot(1,2,2)
sns.stripplot(x = "BsmtHalfBath", y = "SalePrice", data=train,ax=ax2, jitter=True)
ax2.set_title('BsmtHalfBath - SalePrice')
ax2.legend()

Find:

BsmtFUllBATH의 경우에는 최소값에 영향을 주고 & HalfBath 의 경우에는 salprice에 큰 영향을 주지 않는 것으로 보인다.

#### 2-1-7 TotRm and Bedroom

In [ ]:
plt.figure(figsize=(15,10))
ax1 = plt.subplot(2,1,1)
sns.stripplot(x='TotRmsAbvGrd', y='SalePrice', hue='TotRmsAbvGrd', data=train, jitter=True, alpha=0.6, ax=ax1)

ax2 = plt.subplot2grid((2,2), (1,0))
sns.countplot(x = "TotRmsAbvGrd", data=train , ax = ax2, order = train["TotRmsAbvGrd"].value_counts().index)

ax3 = plt.subplot2grid((2,2), (1,1))
sns.boxplot(x = 'TotRmsAbvGrd', y = 'OverallQual', data = train, ax=ax3)
plt.show()

grid = sns.jointplot(x = "TotRmsAbvGrd", y = "SalePrice", data=train, kind="reg", size = 10)


In [ ]:
plt.figure(figsize=(15,6))

ax1 = plt.subplot(1,2,1)
for TotRmsAbvGrd in range(0,5):
    index = train.TotRmsAbvGrd == TotRmsAbvGrd
    sns.stripplot(x = train.BsmtFullBath, y = train.SalePrice, data=train,ax = ax1, label=TotRmsAbvGrd, alpha='0.5')

ax1.legend()
ax1.set_title('TotRmsAbvGrd - SalePrice')

ax2 = plt.subplot(1,2,2)
sns.stripplot(x = "BedroomAbvGr", y = "SalePrice", data=train,ax=ax2, jitter=True)
ax2.set_title('BedroomAbvGr - SalePrice')
ax2.legend()

grid = sns.jointplot(x = "BedroomAbvGr", y = "SalePrice", data=train, kind="reg", size = 10)


plt.show()

Find: 

확실히 TotRmsAbvGrd 는 연관관계가 있는것으로 보임

In [ ]:
stats.probplot(train["TotRmsAbvGrd"], plot=plt)
stats.probplot(train["BedroomAbvGr"], plot=plt)

In [ ]:
train[['TotRmsAbvGrd', 'BedroomAbvGr']].hist(bins=80, figsize=(10,5))

Tot : 길게 하나 하고, 어떤 방이 여기에 포함되어있는지 확인필요

이런 데이터의 경우에는 확실히 한단계 데이터 조치가 필요할것으로 보임
이런 경우는 어떻게 해야하지?

Full Bath의 경우에는 영향이 있으나 HalfBath의 경우에는 영향이 미미한것으로 확인할 수 있음

### 2-2 Missing Data 처리

3가지 정도의 가설

1. 미싱데이터 80% 이상의 데이터들을 제거했을때
2. 비어있는 값들의 처리를 0 or mean 값으로 한경우
3. one-hot encoding 했을때

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]

all_data = pd.concat((train, test)).reset_index(drop=True)
print("All data size is {}".format(all_data.shape))

In [ ]:
total = all_data.isnull().sum().sort_values(ascending=False)[:30]
ratio = ((all_data.isnull().sum()/len(all_data))*100).sort_values(ascending=False)[:30]
missing_data = pd.concat([total, ratio], axis=1, keys=['Total', 'Ratio'])


f, ax = plt.subplots(figsize=(15,7))
plt.xticks(rotation='45')
sns.barplot(x=ratio.index, y=ratio)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing valurs', fontsize=15)
plt.title('Percent missing data by features', fontsize=15)

In [ ]:
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")

방법 1

In [ ]:
feats=["MasVnrArea", "BsmtUnfSF", "TotalBsmtSF", "GarageCars", "BsmtFinSF2", "BsmtFinSF1", "GarageArea"]
for feat in feats:
    all_data[feat].fillna(0, inplace=True)

In [ ]:
# fill in with None
feats1 = ["PoolQC" , "MiscFeature", "Alley", "Fence", "FireplaceQu", "GarageQual", "GarageCond", "GarageFinish", "GarageYrBlt", "GarageType", "BsmtExposure", "BsmtCond", "BsmtQual", "BsmtFinType2", "BsmtFinType1", "MasVnrType"]
for feat in feats1:
    all_data[feat].fillna("None", inplace=True)

In [ ]:
# fill in with mode
feats2 = ["MSZoning", "BsmtFullBath", "BsmtHalfBath", "Utilities", "Functional", "Electrical", "KitchenQual", "SaleType","Exterior1st", "Exterior2nd"]
for feat in feats2:
    all_data[feat].fillna(all_data[feat].mode()[0], inplace=True)

방법 2

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)

for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

all_data['MSZoning']= all_data["MSZoning"].fillna(all_data['MSZoning'].mode()[0])

all_data['Utilities'] = all_data['Utilities'].fillna(all_data['Utilities'].mode()[0])

all_data['Functional']= all_data["Functional"].fillna("Typ")

all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
all_data["Electrical"] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

all_data["KitchenQual"]=all_data["KitchenQual"].fillna(all_data['KitchenQual'].mode()[0])

all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])

all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

all_data = all_data.drop(['Id'], axis=1) # Id 제거

In [ ]:
train = all_data[:ntrain]
test = all_data[ntrain:]
y_train = train.SalePrice.values
all_data.drop(['SalePrice'], axis=1, inplace=True)

### 2-3 아웃라이어 제거

##### OLS 를 사용하여 아웃라이어 제거

In [ ]:
all_data_dummy = pd.get_dummies(all_data)

In [ ]:
all_data_dummy.drop(["SalePrice"], axis=1, inplace=True) 

In [ ]:
x_train_x0 = all_data_dummy[:ntrain]

x_train = sm.add_constant(x_train_x0)
x_columns = x_train.columns

x_train1 = np.array(x_train)
y_train1 = np.array(y_train)

In [ ]:
#OLS 모델 사용
model = sm.OLS(pd.DataFrame(y_train1, columns=["SalePrice"]),pd.DataFrame(x_train1,columns=x_columns))
result = model.fit()
print(result.summary())

In [ ]:
#잔차 정규성 확인
sp.stats.probplot(result.resid, plot=plt)
plt.show()

#Chi^2 확인
test_chi = sms.omni_normtest(result.resid)
for xi in zip(['Chi^2', 'P-value'], test_chi):
    print("%-12s: %6.3f" % xi)

In [ ]:
# 아웃라이어 제거
idx = result.resid_pearson < 2

train = train[idx]

## 3. 변수선정 

In [ ]:
corrmat = train.corr()

f, ax = plt.subplots(figsize = (19,10))
sns.heatmap(corrmat, vmax =1, square = True)

In [ ]:
cols=['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'SalePrice']

In [ ]:
x_train = train[cols]
x_train.tail()

## 4. 정규화 & 스케일링

In [ ]:
# scaler = StandardScaler()
# scaler.fit(x_train)
# x_train = scaler.transform(x_train)

In [ ]:
# scaler = StandardScaler()
# scaler.fit(x_test)
# x_test = scaler.transform(x_test)

In [ ]:
# y_ltrain = np.log1p(y_train)

## 5. 교차 검증 및 모델링

In [ ]:
# x_train_dummy = pd.get_dummies(x_train)
# x_train_0 = sm.add_constant(x_train_dummy)
# x_columns = x_train_0.columns

# test_dummy = pd.get_dummies(test)
# test_0 = sm.add_constant(test_dummy)
# t_columns = test_0.columns


# x_train_m = np.array(x_train_0)
# test_m = np.array(test_0)
# y_train_m = np.array(y_train)


In [ ]:
train.tail()

## 6. 예측하기

score : 0.18836

3113 / 4396 = 70.8%